In [1]:
# Install Pillow, pytesseract and translate, via Conda or pip
# Install Tesseract on your PC, https://github.com/UB-Mannheim/tesseract/wiki
from pytesseract import pytesseract
from translate import Translator

import re
import string

import cv2
import os

import csv
import pandas as pd
import numpy as np

import re

import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('omw-1.4')

import pickle
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\20190735\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\20190735\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\20190735\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
# Transcribes image to text
def transcribe(img):
    transcription = pytesseract.image_to_string(img)

    return transcription

In [3]:
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = [i.split('\n') for i in text.split(' ')]
    text = [item for sublist in text for item in sublist]
    text = [i for i in text if i not in stopwords.words('english')]
    text = ' '.join(text)
    text = text.lower() 
    text = text.strip()  
    text = re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text)
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    
    # Lemmatize
    text = [lemmatizer.lemmatize(word) for word in text.split()]
    text = ' '.join([i for i in text if len(i) > 1])
    
    return text

In [4]:
def transcribe_folder(img_folder):
    names = []
    texts = []
    unprocessed_texts = []
    for img_name in os.listdir(img_folder):
        img_path = os.path.join(img_folder, img_name)
        
        try:
            img = cv2.imread(img_path)
            names.append(img_path)
            text = transcribe(img)
            unprocessed_texts.append(text)
            text = preprocess(text)
            texts.append(text)
        except TypeError:
            "error"
            
    return names, texts, unprocessed_texts

In [5]:
def classify(texts):
    filename = 'pipeline.sav'
    pipeline = pickle.load(open(filename, 'rb'))
    target = pipeline.predict(texts)
    return target

In [6]:
# Create data
def create_data(img_folder, csv_file):
    names, texts, unprocessed_texts = transcribe_folder(img_folder)
    timestamps = [os.path.basename(i).split('.')[0] for i in names]
    target = classify(texts)
    df = pd.DataFrame()
    df['Timestamp'] = timestamps
    df['Text'] = unprocessed_texts
    df['Target'] = target
    df.to_csv(csv_file, index=False) 

In [7]:
def analyze_instance(instance):
    path_img = os.path.join(r'data\keylogging', instance)
    path_img = os.path.join(path_img, 'images')
    path_c = os.path.join(r'data\keylogging', instance)
    path_c = os.path.join(path_c, 'classification.csv')
    
    create_data(path_img, path_c)

In [8]:
analyze_instance('instance_test')